In [ ]:
"""
This is a preliminary version of the last mile delivery use case
"""
import os
os.chdir(r"C:\workspace3\bluesky-USEPE-github")
import configparser
import pickle
import osmnx as ox
import datetime
import geopandas as gpd
from IPython.display import Image
import random

from nommon.city_model.city_graph import cityGraph
from nommon.city_model.corridors_implementation import corridorLoad
from nommon.city_model.dynamic_segments import dynamicSegments
from nommon.city_model.multi_di_graph_3D import MultiDiGrpah3D
from nommon.city_model.utils import read_my_graphml, layersDict
from nommon.city_model.path_planning import trajectoryCalculation, printRoute
from nommon.city_model.scenario_definition import createFlightPlan
from nommon.city_model.strategic_deconfliction import deconflcitedScenario, initialPopulation
from nommon.wind.wind_preprocess import main

# USE CASE

In [ ]:
use_case_1 = Image(filename=r"G:\Mi unidad\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP2 Management\Meetings\20220124 WP4 technical session - monthly meeting\images\use_case_1.png")
use_case_2 = Image(filename=r"G:\Mi unidad\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP2 Management\Meetings\20220124 WP4 technical session - monthly meeting\images\use_case_3.png")
display(use_case_1)
display(use_case_2)

# Configuration file
We create a config file with the following fields:

* [City]

mode = rectangle

#hannover_lat = 52.376

#hannover_lon = 9.76

#zone_size = 1000

hannover_lat_min = 52.36

hannover_lat_max = 52.40

hannover_lon_min = 9.71

hannover_lon_max = 9.76

import = False

imported_graph_path = ./data/last_mile_delivery_test.graphml

* [Layers]

number_of_layers = 9

layer_width = 25

* [BuildingData]

lat_min = 52.36

lat_max = 52.40

lon_min = 9.71

lon_max = 9.76

divisions = 8

directory_hannover = C:\Users\jbueno\Desktop\Stadtmodell_Hannover_CityGML_LoD1\LoD1_Graph

* [Options]

one_way = False

simplify = False

simplification_distance = 10

* [Outputs]

graph_path = ./data/last_mile_delivery_test.graphml

* [Segments]

import = True

path = ./data/offline_segments.pkl

* [Corridors]

corridors = 1 2 3 4

altitude = 250

delta_z = 25

speed = 100

acceleration_length = 50

file_path_corridors = ./data/usepe-hannover-corridors.geojson

* [Strategic_Deconfliction]

ratio = 3

delay = 60

In [ ]:
    # -------------- 1. CONFIGURATION FILE -----------------
    """
    This section reads the configuration file.
    Change the config_path to read the desired file
    """
    # CONFIG
    config_path = r"C:\workspace3\bluesky-USEPE-github\nommon\use_case\settings_last_mile_delivery_test.cfg"
    config = configparser.ConfigParser()
    config.read( config_path )


# City Graph

In [ ]:
    # -------------- 2. CITY GRAPH -------------------------
    """
    This section creates a city graph or loads the graph defined with the city section of the
    configuration file.
    """
    # City
    if config['City'].getboolean( 'import' ):
        filepath = config['City']['imported_graph_path']
        G = read_my_graphml( filepath )
        G = MultiDiGrpah3D( G )
        # fig, ax = ox.plot_graph( G )
    else:
        G = cityGraph( config )


# Segments

In [ ]:
    # -------------- 3. SEGMENTS ----------------------------
    """
    This section creates a airspace segmentation or loads the segmentation defined with the segment
    section of the configuration file.
    Comment it to neglect the segmentation
    """
    os.chdir(r"C:\workspace3\bluesky-USEPE-github\nommon\use_case")
    if config['Segments'].getboolean( 'import' ):
        path = config['Segments']['path']
        with open( path, 'rb' ) as f:
            segments = pickle.load( f )
    else:
        segments = None
    
    path = r"./data/hannover.geojson"
    cells = gpd.read_file(path, driver="GeoJSON")
    cells.plot(column = "class")
    # G, segments = dynamicSegments( G, config, segments, deleted_segments=None )

# Corridors

In [ ]:
    # -------------- 4. CORRIDORS ---------------------------
    """
    This section loads the corridors defined with the corridor section of the configuration file
    Comment it to neglect the creation of corridors
    """
    G, segments = corridorLoad( G, segments, config )
    # G, segments = dynamicSegments( G, config, segments, deleted_segments=None )


In [ ]:
fig, ax = ox.plot_graph( G )

# Segment update

In [ ]:
    # ---------------- 5. Segment update --------------------
    """
    Once we have loaded the segments and we created the corridors, we update the graph based on the parameters of the segments
    """
    G, segments = dynamicSegments( G, config, segments, deleted_segments=None )

# Path planning


In [ ]:
display(use_case_2)

# Scenario 1
## Route A - B 

In [ ]:
    # -------------- 6. PATH PLANNING -----------------------
    """
    This section computes an optimal trajectory from origin to destination. The trajectory is
    optimal according to travel time.
    Comment it to no calculate an optimal trajectory
    Introduce origin and destination points inside the graph
    """
    A = [9.715, 52.39 ]  # origin point
    B = [9.7493106, 52.377872]  # destination point
    travel_time, route = trajectoryCalculation( G, A, B )
    print( 'The travel time of the route is {0}'.format( travel_time ) )
    # print( 'The route is {0}'.format( route ) )
    fig, ax = ox.plot_graph_route( G, route, node_size=0 )
    ax.set_xlim(9.709,9.761)
    ax.set_ylim(52.359,52.401)
    fig

In [ ]:
print(route)

### BlueSky scenario

In [ ]:
# -------------- 7. Scenario definition -----------------------
"""
This section computes scenarios to be used in BlueSky.
We generate the flight plan of one drone. A scenario file is generated, which can be loaded by
BlueSky. The "createFlightPlan" function transforms the optimal path (list of waypoints) to
BlueSky commands
"""
ac = 'U001'
departure_time = '00:00:00.00'
scenario_path = r'.\scenario\U001_AB.scn'
scenario_file = open( scenario_path, 'w' )
layers_dict = layersDict( config )
createFlightPlan( route, ac, departure_time, G, layers_dict, scenario_file )
scenario_file.close()


In [ ]:
bluesky = Image(filename=r"G:\Mi unidad\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP2 Management\Meetings\20220124 WP4 technical session - monthly meeting\images\BlueSky.png")
display(bluesky)

Write de commands:
* VIS MAP TILEDMAP
* PAN 52.39 9.715
* ZOOM 100

In [ ]:
bluesky2 = Image(filename=r"G:\Mi unidad\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP2 Management\Meetings\20220124 WP4 technical session - monthly meeting\images\BlueSky_2.png")
display(bluesky2)

Running the scenario we have just created:
* IC {path}

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def play(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=1000 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

filename = r"G:\Mi unidad\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP2 Management\Meetings\20220124 WP4 technical session - monthly meeting\videos\scenario1.mp4"
play(filename)

# Scenario 2
## Route C - D

In [ ]:
display(use_case_2)

In [ ]:
    # -------------- 6. PATH PLANNING -----------------------
    """
    This section computes an optimal trajectory from origin to destination. The trajectory is
    optimal according to travel time.
    Comment it to no calculate an optimal trajectory
    Introduce origin and destination points inside the graph
    """
    C = [9.75, 52.394 ]  # origin point
    D = [9.742, 52.367] # destination point
    travel_time, route = trajectoryCalculation( G, C, D )
    print( 'The travel time of the route is {0}'.format( travel_time ) )
    # print( 'The route is {0}'.format( route ) )
    fig, ax = ox.plot_graph_route( G, route, node_size=0 )
    ax.set_xlim(9.709,9.761)
    ax.set_ylim(52.359,52.401)
    fig

In [ ]:
# -------------- 7. Scenario definition -----------------------
"""
This section computes scenarios to be used in BlueSky.
We generate the flight plan of one drone. A scenario file is generated, which can be loaded by
BlueSky. The "createFlightPlan" function transforms the optimal path (list of waypoints) to
BlueSky commands
"""
ac = 'U002'
# departure_time = '00:00:00.00'
departure_time = str( datetime.timedelta( seconds=308 ) )
scenario_path = r'.\scenario\U002_CD.scn'
scenario_file = open( scenario_path, 'w' )
layers_dict = layersDict( config )
createFlightPlan( route, ac, departure_time, G, layers_dict, scenario_file )
scenario_file.close()

## BlueSky
We create a scenario file "scenario_2.scn" with these two lines:

00:00:00> PCALL ../nommon/use_case/scenario/U001_AB.scn

00:08:36> PCALL ../nommon/use_case/scenario/U002_CD.scn REL

Running this scenario:

In [ ]:
filename = r"G:\Mi unidad\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP2 Management\Meetings\20220124 WP4 technical session - monthly meeting\videos\scenario2_1.mp4"
play(filename)

# Strategic Deconfliction: A - B and C - D

In [ ]:
# -------------- 8. Strategic deconfliction -----------------------
"""
This section computes an strategic deconflicted trajectory from origin to destination. An
empty initial population is generated.
"""
initial_time = 0  # seconds
final_time = 1800  # seconds
users = initialPopulation( segments, initial_time, final_time )
segments['410']['capacity'] = 1

In [ ]:
orig = A  # origin point
dest = B  # destination point
ac = 'U001'
departure_time = 0  # seconds
scenario_path =  r'.\scenario\U001_AB_deconflicted.scn'
scenario_file = open( scenario_path, 'w' )
users, route = deconflcitedScenario( orig, dest, ac, departure_time, G, users, initial_time,
                              final_time, segments, layers_dict, scenario_file, config )
scenario_file.close()

In [ ]:
orig = C  # origin point
dest = D  # destination point
ac = 'U002'
departure_time = 308  # seconds
scenario_path =  r'.\scenario\U002_CD_deconflicted.scn'
scenario_file = open( scenario_path, 'w' )
users, route = deconflcitedScenario( orig, dest, ac, departure_time, G, users, initial_time,
                              final_time, segments, layers_dict, scenario_file, config )
scenario_file.close()

fig, ax = ox.plot_graph_route( G, route, node_size=0 )
ax.set_xlim(9.709,9.761)
ax.set_ylim(52.359,52.401)
cells.filter(items = [410], axis=0).plot( ax = ax, alpha = 0.6)
fig

In [ ]:
filename = r"G:\Mi unidad\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP2 Management\Meetings\20220124 WP4 technical session - monthly meeting\videos\scenario2_2.mp4"
play(filename)

# Strategic Deconfliction: A - B and C - B

In [ ]:
display(use_case_2)

In [ ]:
# -------------- 8. Strategic deconfliction -----------------------
"""
This section computes an strategic deconflicted trajectory from origin to destination. An
empty initial population is generated.
"""
initial_time = 0  # seconds
final_time = 1800  # seconds
users = initialPopulation( segments, initial_time, final_time )
segments['410']['capacity'] = 1

In [ ]:
orig = A  # origin point
dest = B  # destination point
ac = 'U001'
departure_time = 0  # seconds
scenario_path =  r'.\scenario\U001_AB_deconflicted.scn'
scenario_file = open( scenario_path, 'w' )
users, route = deconflcitedScenario( orig, dest, ac, departure_time, G, users, initial_time,
                              final_time, segments, layers_dict, scenario_file, config )
scenario_file.close()

In [ ]:
orig = C  # origin point
dest = B  # destination point
ac = 'U002'
departure_time = 308  # seconds
scenario_path =  r'.\scenario\U002_CB_deconflicted.scn'
scenario_file = open( scenario_path, 'w' )
users, route = deconflcitedScenario( orig, dest, ac, departure_time, G, users, initial_time,
                              final_time, segments, layers_dict, scenario_file, config )
scenario_file.close()

fig, ax = ox.plot_graph_route( G, route, node_size=0 )
ax.set_xlim(9.709,9.761)
ax.set_ylim(52.359,52.401)
cells.filter(items = [410], axis=0).plot( ax = ax, alpha = 0.6)
fig

# N flight plans + conflict detection

In [ ]:
def detectNoFlyZone(lat, lon):
    for segment in segments.keys():
        if (lat > segments[segment]['lat_min']) & \
            (lat < segments[segment]['lat_max']) & \
            (lon > segments[segment]['lon_min']) & \
            (lon < segments[segment]['lon_max']):
            if segments[segment]['class'] == 'white':
                return False
            else:
                return True
    
    return True
    

In [ ]:
# -------------- 8. Strategic deconfliction -----------------------
"""
This section computes an strategic deconflicted trajectory from origin to destination. An
empty initial population is generated.
"""
initial_time = 0  # seconds
final_time = 1800  # seconds
users = initialPopulation( segments, initial_time, final_time )
segments['410']['capacity'] = 3

In [ ]:
N = 20
os.chdir(r"C:\workspace3\bluesky-USEPE-github\nommon\use_case")

scenario_path =  r'.\scenario\N_flights.scn'
scenario_file_N = open( scenario_path, 'w' )

for i in range(1,N+1):
    flag = True
    while flag:
        orig_lat = random.uniform( config['City'].getfloat( 'hannover_lat_min' ), config['City'].getfloat( 'hannover_lat_max' ) )
        orig_lon = random.uniform( config['City'].getfloat( 'hannover_lon_min' ), config['City'].getfloat( 'hannover_lon_max' ) )
        flag = detectNoFlyZone(orig_lat, orig_lon)
    
    orig = [orig_lon, orig_lat]
    
    flag = True
    while flag:
        dest_lat = random.uniform( config['City'].getfloat( 'hannover_lat_min' ), config['City'].getfloat( 'hannover_lat_max' ) )
        dest_lon = random.uniform( config['City'].getfloat( 'hannover_lon_min' ), config['City'].getfloat( 'hannover_lon_max' ) )
        flag = detectNoFlyZone(dest_lat, dest_lon)
        travel_time, route = trajectoryCalculation( G, orig, [dest_lon, dest_lat] )
        if travel_time > 1800:
            flag = True
    
    dest = [dest_lon, dest_lat]
    
    
    ac = 'U' + str(i)
    departure_time = 0  # seconds
    scenario_path =  r'.\scenario\{0}.scn'.format(ac)
    scenario_file = open( scenario_path, 'w' )
    users, route = deconflcitedScenario( orig, dest, ac, departure_time, G, users, initial_time,
                                  final_time, segments, layers_dict, scenario_file, config )
    scenario_file.close()
    
    scenario_file_N.write( '00:00:00.00 > PCALL ' + '../nommon/use_case/scenario/{0}.scn'.format(ac) + ' REL' + '\n' )
    
    
scenario_file_N.close() 

Write de commands:
* VIS MAP TILEDMAP
* PAN 52.39 9.715
* ZOOM 100

Conflict detection:
* ASAS ON
* ZONER 0.02

Load N flight plans:
* IC {path}

In [ ]:
filename = r"G:\Mi unidad\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP2 Management\Meetings\20220124 WP4 technical session - monthly meeting\videos\scenarioN.mp4"
play(filename)

# Wind

In [ ]:
os.chdir(r"C:\workspace3\bluesky-USEPE-github\nommon\use_case")

path = r"..\wind\data\test_hannover_1m_masked_M03.000.nc"
grid_spacing_list = [5, 10, 20, 50]
time = 0

main(path, grid_spacing_list, time)

Import wind:
* PCALL {path} REL

### Results
Simulations are performed as fast as possible. Scenario characteristics are:
* Number of drones = 20
* Spatial resolution = 10m
* Domain size = 1kmx1km

-------------------------------
Maximum timestep without wind: x5.6

Maximum timestep with wind: x1.1

-----
